In [1]:
# Thermostat UI2

In [2]:
import requests
from ipywidgets import Button, HBox, IntProgress, IntSlider, Label, Layout, VBox
import time
import logging
from utils import *
import uuid
import smartpynector as sp

import ipywidgets as widgets
from IPython.display import display
import time

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"
PREFIXES={
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "saref": "https://w3id.org/saref#",
    }

GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                     ?meas saref:hasValue ?temp .
                     ?meas saref:isMeasuredIn saref:TemperatureUnit .
                     ?meas saref:hasTimestamp ?timestamp .
                     ?meas saref:isMeasurementOf ?room_id .
                     ?meas saref:relatesToProperty saref:Temperature .
                     ?meas saref:measurementMadeBy ?device_id ."""

In [4]:
label_actual = widgets.Label(value='', orientation='vertical')
progress = widgets.IntProgress(value=0, min=0, max=50, step=1, description='', bar_style='info', orientation='vertical')
desired_temp_slider = IntSlider(value=0, min=0, max=50, orientation="vertical")
label_desired = widgets.Label(value='', orientation='vertical')
submit_button = Button(description="Submit", button_style="danger")



In [5]:
def handle_react_measurements(bindings):
    for binding in bindings:
        progress.value = binding['temp']
        label_actual.value = f"{ binding['temp'] } ℃"
    return []

In [6]:
ask_measurements_ki =''
def start_ui_kb(kb_id, kb_name, kb_description, ke_endpoint):
    register_knowledge_base(kb_id, kb_name, kb_description, ke_endpoint)

    react_measurements_ki = register_react_knowledge_interaction(
        GRAPH_PATTERN,
        None,
        "react-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )
    
    global ask_measurements_ki
    ask_measurements_ki = register_ask_knowledge_interaction(
        GRAPH_PATTERN,
        "ask-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )


    start_handle_loop(
        {
            react_measurements_ki: handle_react_measurements,
        },
        kb_id,
        ke_endpoint,
    )


In [10]:
desired_temp=0
def update_desired_temp(change):
    global desired_temp
    desired_temp = change.new
    label_desired.value = f"{ change.new } ℃"

def button_click(desired_temp,ask_measurements_ki, kb_id, ke_endpoint):
    pass
#     ask([{
#         "setting": f"<{THERMOSTAT_API_URL}/settings/{str(uuid.uuid4())}>",
#         "timestamp": f'"{get_timestamp_now()}"',  # ISO 8601 format
#         "room_id": f'"{THERMOSTAT_API_URL}/rooms/1"',
#         "device_id": f'"{THERMOSTAT_API_URL}/devices/1"',
#         "desired_temp": f"{desired_temp}",
#         }], ask_measurements_ki, kb_id, ke_endpoint)

# submit_button.on_click(button_click(desired_temp,ask_measurements_ki, "http://example.org/ui", "http://knowledge_engine:8280/rest/",))


In [14]:
display(VBox([ HBox([label_actual,progress,desired_temp_slider,label_desired]),submit_button]))
start_ui_kb(
    "http://example.org/ui",
    "UI",
    "UI for measurement",
    "http://knowledge_engine:8280/rest/",
)

INFO:utils:registered UI
INFO:utils:received issued knowledge interaction id: http://example.org/ui/interaction/react-measurements
INFO:utils:received issued knowledge interaction id: http://example.org/ui/interaction/ask-measurements


KeyboardInterrupt: 